In [1]:
import json
import requests
import sqlite3

In [2]:
bulk_data_endpoint = 'https://api.scryfall.com/bulk-data'

In [3]:
res = json.loads(
    requests.get(bulk_data_endpoint).text
)

In [4]:
en_card_uri = [
    elem for elem in res['data']
    if elem['type'] == 'default_cards'
][0]['download_uri']

print(en_card_uri)

https://data.scryfall.io/default-cards/default-cards-20240401210705.json


In [5]:
def download_file(url):
    # via https://stackoverflow.com/a/16696317
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    return local_filename

In [6]:
local_card_path = download_file(en_card_uri)

In [7]:
with open(local_card_path, 'r') as infile:
    en_cards = json.load(infile)

In [8]:
card_attribs = [
    'id',
    'name',
    'scryfall_uri',
    'type_line',
    'colors',
    'color_identity',
    'cmc',
    'keywords',
    'set',
    'set_name',
    'collector_number',
    'prices'
]

en_cards = [
    {k: card.get(k, None) for k in card_attribs}
    for card in en_cards
]

In [9]:
con = sqlite3.connect('mtg.db')

In [10]:
cur = con.cursor()

In [11]:
_create_table = """
CREATE TABLE IF NOT EXISTS scryfall(
  id TEXT PRIMARY KEY NOT NULL,
  name TEXT NOT NULL,
  scryfall_uri TEXT,
  type_line TEXT,
  cmc REAL,
  set_shortname TEXT NOT NULL,
  set_name TEXT,
  collector_number TEXT NOT NULL
);
"""
cur.execute(_create_table)

_create_table = """
CREATE TABLE IF NOT EXISTS scryfall_prices(
    card_id TEXT PRIMARY KEY NOT NULL,
    usd REAL,
    usd_foil REAL,
    FOREIGN KEY(card_id) REFERENCES scryfall(id)
);
"""
cur.execute(_create_table)

In [12]:
_query = """
    INSERT INTO scryfall(
        id, name, scryfall_uri,
        type_line, cmc, set_shortname,
        set_name, collector_number
    ) VALUES(
        :id, :name, :scryfall_uri,
        :type_line, :cmc, :set,
        :set_name, :collector_number
    )
    ON CONFLICT(id) DO NOTHING;
"""

cur.executemany(_query, en_cards)
con.commit()

In [13]:
_create_index = """
CREATE INDEX IF NOT EXISTS idx_set_cn ON scryfall(
    set_shortname, collector_number
);
"""
cur.execute(_create_index)

In [14]:
prices = [
    {
        'id': card['id'],
        'usd': card['prices']['usd'],
        'usd_foil': card['prices']['usd_foil']
    } for card in en_cards
]

In [15]:
_query = """
    INSERT INTO scryfall_prices(
        card_id, usd, usd_foil
    ) VALUES(
        :id, :usd, :usd_foil
    )
    ON CONFLICT(card_id) DO UPDATE set usd=:usd, usd_foil=:usd_foil;
"""

cur.executemany(_query, prices)
con.commit()

In [16]:
res = cur.execute("SELECT name, set_shortname, collector_number FROM scryfall s where name = 'Venser, Shaper Savant'")

In [17]:
res.fetchall()

[('Venser, Shaper Savant', 'fut', '46'),
 ('Venser, Shaper Savant', 'plst', 'TSR-96'),
 ('Venser, Shaper Savant', '2x2', '435'),
 ('Venser, Shaper Savant', '2x2', '66'),
 ('Venser, Shaper Savant', 'mm3', '55'),
 ('Venser, Shaper Savant', 'v13', '15'),
 ('Venser, Shaper Savant', 'tsr', '96')]